In [1]:
import pandas as pd
#import pandas.io.sql as sqlio
#import sqlalchemy as sql
import psycopg2
#import matplotlib.pyplot as plt
import numpy as np
import os

#from dotenv import load_dotenv
#from matplotlib import colors
#from matplotlib.ticker import PercentFormatter
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
#load_dotenv()

True

In [2]:
file_to_load = "Resources/Natural Gas Marketed Production-StateRankings.csv"
natural_gas_production = pd.read_csv(file_to_load)
natural_gas_production.head()

,Rank,State,"Natural Gas Marketed Production, million cu ft",Note: Rankings are based on the full source data values. Excludes Gulf of Mexico federal offshore production.
0,1,TX,7847102,NaN
1,2,PA,6210673,NaN
2,3,OK,2946117,NaN
3,4,LA,2810636,NaN
4,5,OH,2409153,NaN


In [3]:
states_data['state id']=states_data.index+1
states_data.head()

State  Abbrev Code  state id
0     Alabama    Ala.   AL         1
1      Alaska  Alaska   AK         2
2     Arizona   Ariz.   AZ         3
3    Arkansas    Ark.   AR         4
4  California  Calif.   CA         5

In [4]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Coal Production-StateRankings.csv"
Coal_Production = pd.read_csv(file_to_load)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",Note: Rankings are based on the full source data values.
0,1,WY,304188,NaN
1,2,WV,95365,NaN
2,3,PA,49883,NaN
3,4,IL,49563,NaN
4,5,KY,39567,NaN


In [13]:
Coal_Production.drop(columns=["Note: Rankings are based on the full source data values."],inplace=True)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [14]:
Coal_Production['State id']=Coal_Production.index+1
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [18]:
Coal_Production['State id']=Coal_Production.index+1
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [15]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Crude Oil Production-StateRankings.csv"
CrudeOil_Production = pd.read_csv(file_to_load)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,4395,NaN
1,2,NM,885,NaN
2,3,ND,862,NaN
3,4,CO,471,NaN
4,5,AK,404,NaN


In [16]:

CrudeOil_Production.drop(columns=["Note: Rankings are based on the full source data values. Excludes federal offshore production."],inplace=True)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day"
0,1,TX,4395
1,2,NM,885
2,3,ND,862
3,4,CO,471
4,5,AK,404


In [17]:
CrudeOil_Production['State id']=CrudeOil_Production.index+1
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",State id
0,1,TX,4395,1
1,2,NM,885,2
2,3,ND,862,3
3,4,CO,471,4
4,5,AK,404,5


In [ ]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"


# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.columns

### Cleaning data

In [ ]:
renewable_energy_data.rename(columns={'Unnamed: 0' : 'State'}, inplace=True)
bio_power_potential = renewable_energy_data[['State', 'biopowerSolid_GWh', 'biopowerGaseous_GWh']]
bio_power_potential.head()

In [10]:
Totalenergyproduction = Totalenergyproduction[['Rank', 'State', 'Total Energy Production, trillion Btu']]

In [ ]:
Totalenergyprod_clean = pd.merge(Totalenergyproduction, states_data,how='inner',left_on='State', right_on='Code')

Totalenergyprod_clean = Totalenergyprod_clean[['id','Rank','Total Energy Production, trillion Btu']]

Totalenergyprod_clean = Totalenergyprod_clean.rename(columns = {'id' : 'State_id'})

Totalenergyprod_clean.set_index("State_id", inplace=True)

Totalenergyprod_clean.head()

,Rank,"Total Energy Production, trillion Btu"
State_id,,
44,1,20421
39,2,8987
51,3,7718
37,4,4881
49,5,4770


In [ ]:
natural_gas_production['State_id']=natural_gas_production.index+1

natural_gas_production.drop(columns = ["State","Note: Rankings are based on the full source data values. Excludes Gulf of Mexico federal offshore production."], inplace = True)

natural_gas_production.head()

In [45]:
file_to_load = "Resources/P2.csv"
production_df = pd.read_csv(file_to_load)
production_df.head()

,"Table P2. Primary Energy Production Estimates in Trillion Btu, 2018",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State,Fossil Fuels,NaN,NaN,NaN,NaN,NaN,Nuclear Electric Power,NaN,Renewable Energy,NaN,NaN,NaN,NaN,NaN,Total,NaN
2,NaN,Coal a,NaN,Natural \nGas b,NaN,Crude Oil c,NaN,NaN,NaN,Biofuels d,NaN,Wood and Waste e,NaN,Other f,NaN,NaN,NaN
3,NaN,Trillion Btu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,370.5,NaN,149.6,NaN,33.6,NaN,412.6,NaN,1.7,NaN,170.1,NaN,105.0,NaN,"1,243.0",NaN


In [46]:
production_df.rename(columns ={"Table P2. Primary Energy Production Estimates in Trillion Btu, 2018": "State",
                              "Unnamed: 1":"Fossil Fuels", "Unnamed: 3":"Natural Gas", "Unnamed: 5":"Crude Oil",
                              "Unnamed: 9":"Biofuels", "Unnamed: 7":"Nuclear Electric Power","Unnamed: 11":"Wood and Waste",
                              "Unnamed: 13":"Other", "Unnamed: 15":"Total"}, inplace = True)

production_df.drop(columns = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8','Unnamed: 10', 'Unnamed: 12', 'Unnamed: 14',
                             'Unnamed: 16'], inplace = True)

production_df.drop([0,1,2,3], inplace = True)

production_df.head()

,"Table P2. Primary Energy Production Estimates in Trillion Btu, 2018",Fossil Fuels,Natural Gas,Crude Oil,Nuclear Electric Power,Biofuels,Wood and Waste,Other,Total
4,Alabama,370.5,149.6,33.6,412.6,1.7,170.1,105.0,"1,243.0"
5,Alaska,13.8,375.3,997.4,0.0,(s),7.3,16.8,"1,410.6"
6,Arizona,140.8,(s),0.1,325.1,6.8,8.7,139.5,621.0
7,Arkansas,0.0,600.4,28.6,133.0,9.5,85.2,30.3,887.1
8,California,0.0,228.9,965.3,190.4,35.5,130.5,857.6,"2,408.2"
